## 1) Import libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import json
import requests
import geopandas as gpd

## 2) Read in 311 data from Analyze Boston

In [ ]:
then = datetime.now()
# create a vector of URLs
urls = ["https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/311_service_requests_2015.csv"
        , "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/311_service_requests_2016.csv"
        , "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/311_service_requests_2017.csv"
        , "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv"
        , "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv"
        , "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/script_105774672_20210108153400_combine.csv"
        , "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmppgq9965_.csv"
        , "https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmp53_2kemh.csv"
       ]

In [ ]:
# import each csv
dfs = [pd.read_csv(url) for url in urls]

In [ ]:
# combine into a single data frame
df_311 = pd.concat(dfs, ignore_index=True)

In [ ]:
pd.set_option('display.max_columns',None)
df_311.head(1)

In [ ]:
df_311.shape

In [ ]:
# convert the closed date to a date time format
df_311['closed_dt'] = pd.to_datetime(df_311['closed_dt'])
df_311['open_dt'] = pd.to_datetime(df_311['open_dt'])

In [ ]:
# convert date to simple data not date time and rename for merge
df_311['date'] = pd.to_datetime(df_311['open_dt']).dt.normalize()

In [ ]:
# Trim the date to match the first observed date for the Boston crime dataset and the last observed for the rainfall data
df_311 = df_311[(df_311['date'] >= "2015-06-15") & (df_311['date'] <= "2022-11-13")]

In [ ]:
print('FS :', df_311['date'].min())
print('LS :', df_311['date'].max())

## 3) Read in rainfall data from NOAA

In [ ]:
rf_url = 'https://raw.githubusercontent.com/georgetown-analytics/boston-311/main/01%20data/rainfall_2015_present.csv'
rf = pd.read_csv(rf_url,  index_col=0).reset_index()
rf.head()

In [ ]:
rf['date'] = pd.to_datetime(rf['date'])

In [ ]:
print('Min dt:', rf['date'].min())
print('Max dt:', rf['date'].max())

## 4) Merge Boston 311 and rainfall data

In [ ]:
df_311_rf = pd.merge(df_311, rf, how = 'left', on='date')
print('Number of Rows: ', df_311_rf.shape[0])
print('Number of Columns: ', df_311_rf.shape[1])

In [ ]:
df_311_rf.head(1)

## 5) Read in Boston Census 2020 Block Groups GeoJSON Data from Analyze Boston

In [ ]:
bn_url = "https://data.boston.gov/dataset/c478b600-3e3e-46fd-9f57-da89459e9928/resource/98201cf0-8aa9-4751-a34d-4d45191a3456/download/census2020_blockgroups.geojson"
bcb = gpd.read_file(bn_url)
bcb.head(1)

In [ ]:
bcb = bcb.to_crs("EPSG:4326")

In [ ]:
bcb.plot();

## 6) Read in Boston Census Block Group Data from Analyze Boston

In [ ]:
bc = 'https://data.boston.gov/dataset/7846ff3b-b738-47a3-a57e-19de2c753220/resource/72254c2d-43df-4644-96d2-a84b8c6f97c7/download/census-block-group-data.csv'
# This step drops the header to index 1 and deletes the 0th element of the index
df_bc = pd.read_csv(bc)
df_bc.columns = df_bc.loc[0]
df_bc = df_bc.drop(0).reset_index()
del df_bc['index']

In [ ]:
# Parses out the GEOID from the the Geographic Record Id
df_bc['GRI'] = df_bc['Geographic Record Identifier'].str[-12:]

## 7) Merge census data together

In [ ]:
df_cm = pd.merge(left=df_bc,right=bcb,how='left',left_on='GRI',right_on='GEOID20')
print('Number of Rows: ', df_cm.shape[0])
print('Number of Columns: ', df_cm.shape[1])

In [ ]:
df_cm.head(1)

## 8) Read in crime data from Analyze Boston and process UCR part one and two aggregates and total count

In [ ]:
crime_urls = ['https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/792031bf-b9bb-467c-b118-fe795befdf00/download/crime-incident-reports-2015.csv',
              'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/b6c4e2c3-7b1e-4f4a-b019-bef8c6a0e882/download/crime-incident-reports-2016.csv',
              'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/64ad0053-842c-459b-9833-ff53d568f2e3/download/crime-incident-reports-2017.csv',
              'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/e86f8e38-a23c-4c1a-8455-c8f94210a8f1/download/crime-incident-reports-2018.csv',
              'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/34e0ae6b-8c94-4998-ae9e-1b51551fe9ba/download/tmp9mkqyv6b.csv',
              'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/be047094-85fe-4104-a480-4fa3d03f9623/download/script_113631134_20210423193017_combine.csv',
              'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/f4495ee9-c42c-4019-82c1-d067f07e45d2/download/tmp7_f32p54.csv',
              'https://data.boston.gov/dataset/6220d948-eae2-4e4b-8723-2dc8e67722a3/resource/313e56df-6d77-49d2-9c49-ee411f10cf58/download/tmpdg9fc0p7.csv']

In [ ]:
# import each csv
crime_dfs = [pd.read_csv(url) for url in crime_urls]

In [ ]:
# combine into a single data frame
df_crime = pd.concat(crime_dfs, ignore_index=True)

In [ ]:
df_crime = df_crime[(df_crime['UCR_PART'] == 'Part One') | (df_crime['UCR_PART'] == 'Part Two')]

In [ ]:
df_crime['UCR_PART'].value_counts()

In [ ]:
df_crime['ucr_one'] = df_crime['UCR_PART'].apply(lambda x: '1' if x == 'Part One' else '')

In [ ]:
df_crime['ucr_two'] = df_crime['UCR_PART'].apply(lambda x: '2' if x == 'Part Two' else '')

In [ ]:
df_crime['ucr_one'] = pd.to_numeric(df_crime['ucr_one'])
df_crime['ucr_two'] = pd.to_numeric(df_crime['ucr_two'])

In [ ]:
# Make sure the coordinate reference system (crs) is set to the same value as shown in bcb.crs. In this case CRS: EPSG:4326
cgdf = gpd.GeoDataFrame(df_crime, geometry=gpd.points_from_xy(df_crime['Long'], df_crime['Lat']), crs='EPSG:4326')

In [ ]:
cgdf.crs

In [ ]:
# Clip to City of Boston political boundary to strip away bad coordinates to avoid error: Coordinates must not have minimums more than maximums
cgdf_j = gpd.clip(cgdf, bcb)
# Shape of the geodataframe within Boston city limits 
print('Number of Rows: ', cgdf_j.shape[0])
print('Number of Columns: ', cgdf_j.shape[1])

In [ ]:
cgdf_j = gpd.sjoin(cgdf_j, bcb, how='left')
# Shape of the geodataframe within Boston Census Block Groups
print('Number of Rows: ', cgdf_j.shape[0])
print('Number of Columns: ', cgdf_j.shape[1])

In [ ]:
cgdf_j.head(1)

In [ ]:
c_cj = cgdf_j.groupby(['GEOID20']).agg({'ucr_one':'sum','ucr_two':'sum'}).reset_index()

In [ ]:
c_cj['total_ucr'] = c_cj['ucr_one'] + c_cj['ucr_two']

In [ ]:
c_cj.head()

## 9) Merge UCR Crime Count Data to Boston Census Block Group Data

In [ ]:
df_ccm = pd.merge(left=df_cm,right=c_cj,how='left',left_on='GEOID20',right_on='GEOID20')
print('Number of Rows: ', df_ccm.shape[0])
print('Number of Columns: ', df_ccm.shape[1])

In [ ]:
df_ccm.head(1)

In [ ]:
# Convert df_ccm from dataframe to geodataframe
df_ccm = gpd.GeoDataFrame(df_ccm, geometry='geometry')

In [ ]:
# Verify the crs is EPSG: 4326
df_ccm.crs

## 10) Merge all datasets together

In [ ]:
# Make sure the coordinate reference system (crs) is set to the same value as shown in df_ccm.crs. In this case CRS: EPSG:4326
df_311_rf = gpd.GeoDataFrame(df_311_rf, geometry=gpd.points_from_xy(df_311_rf['longitude'], df_311_rf['latitude']), crs='EPSG:4326')

In [ ]:
master = gpd.sjoin(df_311_rf, df_ccm, how='left')
# Shape of the master dataframe
print('Number of Rows: ', master.shape[0])
print('Number of Columns: ', master.shape[1])

In [ ]:
master.head(1)

In [ ]:
now = datetime.now()
duration = now - then
print("Holy cow! I hope you got coffee during the runtime:",duration)